In [1]:
import time
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
driver=webdriver.Chrome() 
website="https://www.techstars.com/newsroom"  
driver.get(website)

all the required path are given below 

In [ ]:
# //*[@id="__next"]/div/div[2]/div[2]/div[1]
# //*[@id="__next"]/div/div[2]/div[2]/div[1]/div/a[1]
# //*[@id="__next"]/div/div[2]/div[2]/div[1]/div/a[2]
# //*[@id="__next"]/div/div[2]/div[2]/div[1]/div/a[3]


# //*[@id="__next"]/div/div[2]/div[2]/div[2]/div
# //*[@id="__next"]/div/div[2]/div[2]/div[2]/div/a[1]..datetime
# //*[@id="__next"]/div/div[2]/div[2]/div[2]/div/a[2]...title
# //*[@id="__next"]/div/div[2]/div[2]/div[2]/div/a[3]....body(content)


start the scraping of the website on the first page 

In [3]:

# Find all articles
articles = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div[2]/div[2]/div')

# Create and open CSV file
csv_filename = "techstars_articles.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Date", "Title", "Content"])  # CSV headers

    for article in articles:
        try:
            date = article.find_element(By.XPATH, './div/a[1]').text.strip()
            title = article.find_element(By.XPATH, './div/a[2]').text.strip()
            content = article.find_element(By.XPATH, './div/a[3]').text.strip()

            writer.writerow([date, title, content])
            print(f" Saved: {title}")

        except Exception as e:
            print(f" Error extracting article: {e}")


print(f"\n Scraping complete! Data saved to {csv_filename}")


 Saved: Techstars and Merce Anders Equity Group Partner to Launch Startup Community Catalyst Africa in Botswana
 Saved: Techstars and Emory University Launch a Pre-Accelerator Program to Drive Entrepreneurial Growth
 Saved: Techstars New York City powered by J.P. Morgan Announces the Fall 2024 Class
 Saved: Better for Founders, Better for Colorado
 Saved: Meet the Techstars Economic Mobility Powered by Samvid Ventures Class of 2024
 Error extracting article: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div/a[1]"}
  (Session info: chrome=131.0.6778.266); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF61BD880D5+2992373]
	(No symbol) [0x00007FF61BA1BFD0]
	(No symbol) [0x00007FF61B8B590A]
	(No symbol) [0x00007FF61B90926E]
	(No symbol) [0x00007FF61B90955C]
	(No symbol) [0x00007FF61B8FC6CC]
	(No symbol) [0x00007FF61B

scrape full data up to 2 month from now

In [4]:
import time
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Initialize WebDriver
driver = webdriver.Chrome()
url = "https://www.techstars.com/newsroom"
driver.get(url)

time.sleep(3)  # Allow initial page to load

# Define time limit (2 months ago)
two_months_ago = datetime.now() - timedelta(days=60)

# Open CSV file to save scraped data
csv_filename = "final_techstars_scraped_articles.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Date", "Title", "Body", "URL"])  # CSV Header

    stop_scraping = False  # Flag to stop scraping when old articles are found

    while not stop_scraping:  # Loop through multiple pages
        # Wait for articles to load
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[2]/div'))
            )
        except TimeoutException:
            print(" Page took too long to load. Stopping.")
            break

        # Extract all articles on the current page
        articles = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div[2]/div[2]/div')
        print(f" Found {len(articles)} articles on this page")

        for index, article in enumerate(articles):
            try:
                # Extract article details
                date_text = article.find_element(By.XPATH, './div/a[1]').text.strip()
                title = article.find_element(By.XPATH, './div/a[2]').text.strip()
                body = article.find_element(By.XPATH, './div/a[3]').text.strip()
                url = article.find_element(By.XPATH, './div/a[2]').get_attribute("href")  # Extract URL

                # Convert date format: 'Jan 06, 2025' → '06, 01, 2025'
                try:
                    article_date = datetime.strptime(date_text, "%b %d, %Y")
                    formatted_date = article_date.strftime("%d, %m, %Y")
                except ValueError:
                    print(f" Invalid date format: {date_text}. Skipping article.")
                    continue

                # Stop scraping immediately if we reach an old article
                if article_date < two_months_ago:
                    print(f"\n Stopping: Found an old article from {formatted_date}.")
                    stop_scraping = True  # Set flag to exit loop
                    break  # Stop processing this page

                # Save article data
                writer.writerow([formatted_date, title, body, url])
                print(f" Saved Article {index+1}: {title} ({formatted_date})")

            except NoSuchElementException:
                print(f" Missing data in article {index+1}. Skipping.")
                continue

        if stop_scraping:
            break  # Exit the while loop after processing the last valid page

        # Try to click the "Next Page" button
        try:
            next_button = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[2]/div[6]/div[7]/button/span[1]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)  # Scroll into view
            time.sleep(2)  # Allow smooth scrolling
            driver.execute_script("arguments[0].click();", next_button)  # Click via JavaScript
            print("➡️ Clicked 'Next Page' button")
            time.sleep(5)  # Wait for new articles to load
        except NoSuchElementException:
            print("\n No more pages found. Stopping.")
            break

print(f"\n Scraping complete! Data saved to {csv_filename}")


 Found 6 articles on this page
 Saved Article 1: Techstars and Merce Anders Equity Group Partner to Launch Startup Community Catalyst Africa in Botswana (13, 01, 2025)
 Saved Article 2: Techstars and Emory University Launch a Pre-Accelerator Program to Drive Entrepreneurial Growth (06, 01, 2025)
 Saved Article 3: Techstars New York City powered by J.P. Morgan Announces the Fall 2024 Class (03, 12, 2024)

 Stopping: Found an old article from 29, 10, 2024.

 Scraping complete! Data saved to final_techstars_scraped_articles.csv
